# GPT训练
    - 无监督训练：以自回归的方式，基于transformer解码器原理，采用掩码自注意力机制,基于字节对编码算法（byte-pair Encoding）,将文本分割为小单元    
    - 有监督微调：根据下游任务(如文本分类、问答、摘要)调整模型参数，使其具有特定领域能力,联合优化任务损失和训练损失
# 解码策略
    - 贪心算法：每次预测都取概率最大的.方法比较简单，但结果全是相同的。只考虑当前最优解，忽视全局最优解.  
        --top K:从概率超过K的中，随机选择一个.,top P方法优化  
        --top P:采用累计分布概率，设定P值,选择累积概率不超过P的序列
        --温度调节:每一个词生产一个logits,使用logits/tem,用来控制logits的分布，tem越高，则越有创意，tem越低则越保守
    - 束搜索：生成过程，
        --有一个beam size=K，每次都在选择K长度的序列，计算联合概率

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as fc

class MiniGPT(nn.Module):
    def __init__(self,vocab_size = 10000,embed_dim = 128,num_layer = 3,max_len = 512):
        super().__init__()
        self.token_embed = nn.Embedding(vocab_size,embed_dim)
        self.pos_embed = nn.Embedding(max_len,embed_dim)
        # 使用编码器层代替GPT解码器
        # pass

        # 最小改动解码器，实现GPT
        self.layers = nn.ModuleList([
                                    nn.TransformerDecoderLayer(
                                        d_model=embed_dim,
                                        nhead=4,
                                        dim_feedforward=embed_dim*4,
                                        dropout = 0.1,
                                        batch_first = True,                        
                                    ) for _ in range(num_layer)
                                    ])
        # 为什么使用ModuleList？使用普通列表，无法将所有参数绑定到类中，使用ModuleList可以将所有可训练parameter()收集
        # 语言模型的输出层
        self.lm_out  = nn.Linear(embed_dim,vocab_size)
        # 分类层：用于模型微调时添加
        self.classifier = None

    def forward(self,input_ids):
        # input_ids.shape = (batch_size,seq_len)
        seq_len = input_ids.size(1)
        pos = torch.arange(seq_len,device=input_ids.device)
        # self.token_embed(input_ids).shape = (seq_len,embed_dim)
        x  = self.token_embed(input_ids)+self.pos_embed(pos)
        mask = nn.Transformer.generate_square_subsequent_mask(seq_len,device=input_ids.device)
        for layer in self.layers:
            x = layer(tgt=x,memory = x,tgt_mask = mask)
        # 语言模型输出
        logits = self.lm_out(x)
        # 分类任务输出
        if self.classifier is not None:
            # 取序列中的首token,作为分类
            cls_logits = self.classifier(x[:,0,:])
            return logits,cls_logits
        return logits

In [3]:
def test():
    vocab_size = 100
    embed_dim = 64
    n_layer = 6
    max_len = 16
    batch_size = 2
    seq_len = 4

    model = MiniGPT(vocab_size,embed_dim,n_layer,max_len)
    input_ids = torch.randint(0,vocab_size,(batch_size,seq_len))
    logits = model(input_ids)
    print('语言模型的形状',logits.shape)
    model.classifier = nn.Linear(embed_dim,3)
    logits,cls_logits = model(input_ids)
    print('分类logits',cls_logits.shape)
test()

语言模型的形状 torch.Size([2, 4, 100])
分类logits torch.Size([2, 3])


# 数据集类

In [21]:
import re
import numpy as np
from torch.utils.data import Dataset,DataLoader
from transformers import AutoTokenizer
from tqdm import tqdm

In [31]:
class TextDataset(Dataset):
    '''文本数据集类，用于预训练模型'''
    def __init__(self,file_path,tokenizer,block_size = 512,stride = 384):
        '''
        parameter
        ------------
        tokenizer:AotoTonkenizer
            分词器
        block_size:int
            分块大小
        stride:int
            滑动的步长
        '''
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.stride = stride

        with open(file_path,mode = 'r',encoding='utf-8') as f:
            text = self.clear_text(f.read())
        self.tokens = self.tokenizer.encode(text,add_special_tokens = True)
        # 若数据量很大，使用动态截取的方式，而不是事先全部计算

    def clear_text(self,text):
        # 段落间的分隔符
        token = self.tokenizer.sep_token + self.tokenizer.cls_token
        text = text.strip()
        text = re.sub('\n',token,text)
        text = re.sub(r'\s+','',text)
        return text
    
    def __len__(self):
        # 计算总样本数:（总长度-块长度）/步长+1
        return (len(self.tokens)-self.block_size)//self.stride+1
    
    def __getitem__(self,idx):
        start = idx * self.stride
        end = start + self.block_size
        # 动态截取token块。
        chunk = self.tokens[start:end]
        x = torch.tensor(chunk[:-1],dtype = torch.long)
        y = torch.tensor(chunk[1:],dtype = torch.long)
        return x,y
    


# 预训练

In [20]:
tokenizer = AutoTokenizer.from_pretrained('uer/gpt2-chinese-cluecorpussmall')

In [35]:
block_size = 64
stride = 64
batch_size = 64
embed_dim = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 50
def pretrain():
    dataset = TextDataset(
        file_path=r'E:\LMAI_study\05 深度学习\data\天龙八部分类.txt',
        tokenizer=tokenizer,
        block_size=block_size,
        stride=stride
    )

    train_loader = DataLoader(dataset,batch_size=batch_size,shuffle = True)

    model = MiniGPT(vocab_size=tokenizer.vocab_size,embed_dim=embed_dim).to(device)
    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        train_loss = 0
        bar = tqdm(train_loader)
        for input,target in bar:
            input ,target = input.to(device) ,target.to(device)
            logits =  model(input)
            # 对不连续的变量改变形状，不能直接用view,可以用reshape,或调用contiguous方法
            loss = criterion(logits.view(-1,tokenizer.vocab_size),target.contiguous().view(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss+=loss.item()
            # 训练一个批次后，显示当前批次的损失
            bar.set_description(f'epoch:{epoch+1} train loss:{loss.item():.4f}')
        train_loss /= len(train_loader)
        print(train_loss)
    torch.save(model.state_dict(),'model.pth')

pretrain()

epoch:1 train loss:9.6956: 100%|██████████| 2/2 [00:00<00:00,  4.92it/s] 


9.903632164001465


epoch:2 train loss:9.0865: 100%|██████████| 2/2 [00:00<00:00,  9.45it/s]


9.212864398956299


epoch:3 train loss:8.6985: 100%|██████████| 2/2 [00:00<00:00,  9.98it/s]


...

1.118008553981781


epoch:49 train loss:0.9934: 100%|██████████| 2/2 [00:00<00:00,  9.31it/s]


1.0414519309997559


epoch:50 train loss:0.9763: 100%|██████████| 2/2 [00:00<00:00,  9.88it/s]

0.9846748113632202
